<a href="https://colab.research.google.com/github/amanjain252002/Tic-Tac-Toe/blob/main/Tic_Tac_Toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import random
import numpy as np
import tensorflow as tf

In [71]:
def Display_Board(Arr, i):
  print("Move No: " + str(i))
  print()
  print(" #", Arr[0][0], "|", Arr[0][1], "|", Arr[0][2], "#")
  print("---------------")
  print(" #", Arr[1][0], "|", Arr[1][1], "|", Arr[1][2], "#")
  print("---------------")
  print(" #", Arr[2][0], "|", Arr[2][1], "|", Arr[2][2], "#")

In [72]:
def Toss():
  Number = random.randint(1, 2)
  if(Number == 1):
    return "Computer"
  else:
    return "Human"

In [73]:
def Row_Check(A):
  Winner = "No Body"
  for i in range(3):
    if((A[i][0] == A[i][1]) and (A[i][0] == A[i][2])):
      if(A[i][0] == 'X'):
        Winner = "Human"
        return True, Winner
      elif(A[i][0] == 'O'):
        Winner = "Computer"
        return True, Winner
  return False, Winner

In [74]:
def Column_Check(A):
  Winner = "No Body"
  for i in range(3):
    if((A[0][i] == A[1][i]) and (A[0][i] == A[2][i])):
      if(A[0][i] == 'X'):
        Winner = "Human"
        return True, Winner
      elif(A[0][i] == 'O'):
        Winner = "Computer"
        return True, Winner
  return False, Winner

In [75]:
def Digonal_Check(A):
  Winner = "No Body"
  if((A[0][0] == A[1][1]) and (A[0][0] == A[2][2])):
    if(A[0][0] == 'X'):
      Winner = "Human"
      return True, Winner
    elif(A[0][0] == 'O'):
      Winner = "Computer"
      return True, Winner
  elif((A[0][2] == A[1][1]) and (A[0][2] == A[2][0])):
    if(A[0][2] == 'X'):
      Winner = "Human"
      return True, Winner
    elif(A[0][2] == 'O'):
      Winner = "Computer"
      return True, Winner
  return False, Winner

In [76]:
def Check_Win(Arr):
  Game_Over = False
  Winner = "No Body"
  Game_Over_1, Winner_1 = Row_Check(Arr)
  Game_Over_2, Winner_2 = Column_Check(Arr)
  Game_Over_3, Winner_3 = Digonal_Check(Arr)
  if(Game_Over_1 or Game_Over_2 or Game_Over_3):
    Game_Over = True
    if(Game_Over_1):
      Winner = Winner_1
    elif(Game_Over_2):
      Winner = Winner_2
    else:
      Winner = Winner_3
  return Game_Over, Winner

In [77]:
def Training_Bot(Arr):
  X = random.randint(0, 2)
  Y = random.randint(0, 2)
  while(Arr[X][Y] != '0'):
    X = random.randint(0, 2)
    Y = random.randint(0, 2)
  Arr[X][Y] = 'X'
  return Arr

In [78]:
def Player_Move(Arr):
  X, Y = list(map(int, input().split()))
  Arr[X][Y] = 'X'
  return Arr

In [79]:
model = tf.keras.models.Sequential([
                                      tf.keras.layers.Dense(64, input_shape = [9], activation = 'relu'),
                                      tf.keras.layers.Dense(128, activation = 'relu'),
                                      tf.keras.layers.Dense(128, activation = 'relu'),
                                      tf.keras.layers.Dense(128, activation = 'relu'),
                                      tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [80]:
model.compile(optimizer = 'adam', loss = 'BinaryCrossentropy', metrics = 'BinaryCrossentropy')

In [81]:
def Computer_Move(Arr, model):
  Input = sum(Arr, [])
  Possible_Inputs = []
  for i in range(9):
    Temp = Input.copy()
    if(Input[i] == '0'):
      Temp[i] = 'O'
      Possible_Inputs.append(Temp)
  for i in range(len(Possible_Inputs)):
    for j in range(len(Possible_Inputs[0])):
      if(Possible_Inputs[i][j] == 'X'):
        Possible_Inputs[i][j] = -1
      elif(Possible_Inputs[i][j] == 'O'):
        Possible_Inputs[i][j] = 1
      else:
        Possible_Inputs[i][j] = 0
  Prediction = model.predict(Possible_Inputs)
  Position = np.argmax(Prediction)
  Next = Possible_Inputs[Position]
  Next_Outcome = []
  for i in range(3):
    Temp = []
    for j in range(3):
      if(Next[3*i + j] == 1):
        Temp.append('O')
      elif(Next[3*i + j] == -1):
        Temp.append('X')
      else:
        Temp.append('0')
    Next_Outcome.append(Temp)
  return Next_Outcome

In [82]:
Next_Outcome = Computer_Move([['0', '0', 'O'], ['O', '0', 'O'], ['X', 'X', '0']], model)

In [83]:
Next_Outcome

[['0', 'O', 'O'], ['O', '0', 'O'], ['X', 'X', '0']]

In [84]:
def Training(model, epochs):
  Win = 0
  Lose = 0
  Draw = 0
  for i in range(epochs):
    Train_X = []
    Train_Y = []
    Arr = []
    for i in range(3):
      Arr.append(['0', '0', '0'])
    i = 1
    Game_Over = False
    Start = Toss()
    while((Game_Over != True) and i < 9):
      if(Start == 'Computer'):
        Arr = Computer_Move(Arr, model)
        Train_X.append(sum(Arr, []))
        i = i + 1
        Game_Over, Winner = Check_Win(Arr)
        if(Game_Over == False):
          Arr = Training_Bot(Arr)
          Game_Over, Winner = Check_Win(Arr)
          Train_X.append(sum(Arr, []))
          i = i + 1
      else:
        Arr = Training_Bot(Arr)
        Game_Over, Winner = Check_Win(Arr)
        Train_X.append(sum(Arr, []))
        i = i + 1
        if(Game_Over == False):
          Arr = Computer_Move(Arr, model)
          Train_X.append(sum(Arr, []))
          i = i + 1
          Game_Over, Winner = Check_Win(Arr)

    for i in range(len(Train_X)):
      for j in range(len(Train_X[i])):
        if(Train_X[i][j] == '0'):
          Train_X[i][j] = 0
        elif(Train_X[i][j] == 'X'):
          Train_X[i][j] = -1
        else:
          Train_X[i][j] = 1
    if(Winner == "Draw"):
      for i in range(len(Train_X)):
        Train_Y.append(0)
      Draw = Draw + 1
    elif(Winner == 'Human'):
      for i in range(len(Train_X)):
        Train_Y.append(0)
      Lose = Lose + 1
    else:
      for i in range(len(Train_X)):
        Train_Y.append(1)
      Win = Win + 1
    model.fit(Train_X, Train_Y, epochs = 2)
  return model, Win, Lose, Draw

In [ ]:
model, Win, Lose, Draw = Training(model, 100)

In [88]:
print(Win, Lose, Draw)

81 19 0


In [89]:
def Game():
  Train_X = []
  Train_Y = []
  Arr = []
  for i in range(3):
    Arr.append(['0', '0', '0'])
  i = 1
  Game_Over = False
  Display_Board(Arr,  0)

  Start = Toss()
  while((Game_Over != True) and i < 9):
    if(Start == 'Computer'):
      Arr = Computer_Move(Arr, model)
      Train_X.append(sum(Arr, []))
      Display_Board(Arr, i)
      i = i + 1
      Game_Over, Winner = Check_Win(Arr)
      if(Game_Over == False):
        Arr = Player_Move(Arr)
        Game_Over, Winner = Check_Win(Arr)
        Display_Board(Arr, i)
        Train_X.append(sum(Arr, []))
        i = i + 1
    else:
      Arr = Player_Move(Arr)
      Game_Over, Winner = Check_Win(Arr)
      Display_Board(Arr, i)
      Train_X.append(sum(Arr, []))
      i = i + 1
      if(Game_Over == False):
        Arr = Computer_Move(Arr, model)
        Train_X.append(sum(Arr, []))
        Display_Board(Arr, i)
        i = i + 1
        Game_Over, Winner = Check_Win(Arr)

  if(Game_Over == False):
    print("Draw")
  elif(Game_Over == True):
    print(Winner)

  for i in range(len(Train_X)):
    for j in range(len(Train_X[i])):
      if(Train_X[i][j] == '0'):
        Train_X[i][j] = 0
      elif(Train_X[i][j] == 'X'):
        Train_X[i][j] = -1
      else:
        Train_X[i][j] = 1
  if(Winner == "Draw"):
    for i in range(len(Train_X)):
      Train_Y.append(0)
  elif(Winner == 'Human'):
    for i in range(len(Train_X)):
      Train_Y.append(0)
  else:
    for i in range(len(Train_X)):
      Train_Y.append(1)
  model.fit(Train_X, Train_Y, epochs = 1)

In [90]:
Game()

Move No: 0

 # 0 | 0 | 0 #
---------------
 # 0 | 0 | 0 #
---------------
 # 0 | 0 | 0 #
1 1
Move No: 1

 # 0 | 0 | 0 #
---------------
 # 0 | X | 0 #
---------------
 # 0 | 0 | 0 #
Move No: 2

 # 0 | 0 | 0 #
---------------
 # 0 | X | 0 #
---------------
 # 0 | 0 | O #
0 1
Move No: 3

 # 0 | X | 0 #
---------------
 # 0 | X | 0 #
---------------
 # 0 | 0 | O #
Move No: 4

 # O | X | 0 #
---------------
 # 0 | X | 0 #
---------------
 # 0 | 0 | O #
2 1
Move No: 5

 # O | X | 0 #
---------------
 # 0 | X | 0 #
---------------
 # 0 | X | O #
Human
1/1 [==============================] - 0s 1ms/step - loss: 1.2446 - binary_crossentropy: 1.2446
